**About** : This notebook is used to train models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import cudf
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *
from numerize.numerize import numerize

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from training.main import k_fold

from utils.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    init_neptune
)

from params import *
from utils.plots import *
from data.dataset import *
from data.preparation import *
from utils.torch import init_distributed, count_parameters, load_model_weights
from utils.plots import plot_sample, plot_sample_with_edges
from model_zoo.models import *

from data.transforms import flip

## Data

In [ ]:
# df = prepare_data(DATA_PATH, processed_folder="torch_3/")
df = prepare_data(DATA_PATH, processed_folder="torch_3/")

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv("../input/folds_4.csv")
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [ ]:
dataset = SignDataset(df, max_len=None, train=False)

In [ ]:
data = dataset[0]

In [ ]:
plot_sample_with_edges(data, n_frames=4, figsize=(10, 10), show_text=False)

In [ ]:
4718

### Lens

In [ ]:
lens = []
for i in tqdm(range(0, len(dataset), 10)):
    data = dataset[i]
    lens.append(len(data['x']))

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x=lens)

In [ ]:
lens = []
for i in tqdm(range(0, len(dataset), 10)):
    data = dataset[i]
    lens.append(len(data['x']))

### 4718

In [ ]:
df = prepare_data(DATA_PATH, processed_folder="torch_3/")

df = df[df['participant_id'] == 4718].reset_index(drop=True)
# df = df[df['sign'] == "pizza"]

In [ ]:
dataset = SignDataset(df, max_len=None, train=False)

In [ ]:
lens = []
for i in tqdm(range(0, len(dataset))):
    data = dataset[i]
    lens.append(len(data['x']))

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x=lens)

### Two hands

In [ ]:
# df = prepare_data(DATA_PATH, processed_folder="torch_3/")
df = prepare_data(DATA_PATH, processed_folder="torch_3/")

In [ ]:
df = df[df['participant_id'] == 29302].reset_index(drop=True)
# df = df[df['sign'] == "pizza"]

In [ ]:
dataset = SignDataset(df, max_len=None, train=False, aug_strength=1)
# dataset = SignDataset(df, max_len=None, train=False)

In [ ]:
two_hands = []
for i in tqdm(range(len(dataset))):
    data = dataset[i]
    
    lh = data['x'].T[data['type'][0] == 5].T
    rh = data['x'].T[data['type'][0] == 10].T

    n_lh = ((lh.abs().max(1).values) > 0).sum()
    n_rh = ((rh.abs().max(1).values) > 0).sum()
    
    n_frames = len(data['x'])
    
    
    
    if n_lh / n_frames > 0.25 and n_rh / n_frames > 0.25:
        two_hands.append(df['sequence_id'][i])
#         if np.random.random() < 0.01:
#         plot_sample_with_edges(data, n_frames=4, figsize=(10, 10))
        
#     if i > 10:
#         break

In [ ]:
len(df), len(two_hands)

In [ ]:
# np.save("../output/two_hands_others.npy", two_hands)
# np.save("../output/two_hands_29302.npy", two_hands)

In [ ]:
df_two_hands = df[df['sequence_id'].isin(two_hands)].reset_index()

In [ ]:
dataset = SignDataset(df_two_hands, max_len=None, train=False, aug_strength=1)

In [ ]:
for i in tqdm(range(len(dataset))):
    print(df_two_hands['sign'][i])
    data = dataset[i]
    
#     print(df['sequence_id'][i], df['sign'][i])
    plot_sample_with_edges(data, n_frames=4, figsize=(10, 10))
    
    if i > 30:
        break

In [ ]:
# plot_sample_with_edges(flip(data), n_frames=2, figsize=(20, 10), show_text=True)

In [ ]:
# lens = []
# for i in tqdm(range(len(df.head(1000)))):
#     lens.append(np.load(dataset.paths[i]).shape[0])
# sns.histplot(lens)

# plt.title((np.array(lens) > 50).mean())
# plt.axvline(50, c="salmon")
# plt.show()

## Model

In [ ]:
model = define_model("mlp_bert_3", transfo_heads=8, embed_dim=16, dense_dim=384, transfo_dim=384, transfo_layers=4, n_landmarks=116)  #, pretrained_weights="../logs/pretrain/2023-03-23/4/mlp_bert_0.pt").cpu()

In [ ]:
numerize(count_parameters(model))

In [ ]:
# model = define_model("gcn", transfo_heads=8, embed_dim=32, transfo_dim=128, transfo_layers=4, n_landmarks=100)  #, pretrained_weights="../logs/pretrain/2023-03-23/4/mlp_bert_0.pt").cpu()

In [ ]:
# model.type_embed.weight.data.fill_(0.)

In [ ]:
dataset = SignDataset(df, max_len=40, train=False)

BS = 32

x = {}
batch = [dataset[idx] for idx in range(BS)]  # 
# batch = [dataset[idx] for idx in np.random.randint(len(dataset), size=BS)]
for k in data:
    x[k] = torch.cat([d[k].unsqueeze(0) for d in batch])  # .cuda()

In [ ]:
y, _ = model(x)
y.size(), y.max()

In [ ]:
# opt = torch.optim.Adam(model.parameters())
# loss = y.mean()
# loss.backward()
# opt.step()
# for name, param in model.named_parameters():
#     if param.grad is None:
#         print(name)

## Training

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    processed_folder = "torch_3/"
    max_len = 40
    resize_mode = "pad"
    aug_strength = 1
    use_extra_data = False
    n_landmarks = 100 if processed_folder != "torch_5/" else 121

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
    name = "gcn"
#     name = "mlp_bert_2"
#     name = "bi_bert"
    pretrained_weights = None  # "../logs/2023-03-27/19/mlp_bert_2_0.pt"  "../logs/pretrain/2023-03-23/4/mlp_bert_0.pt" 
    syncbn = False
    num_classes = 250
    num_classes_aux = 0

    transfo_layers = 4
    embed_dim = 64
    transfo_dim = 64  # 288
    transfo_heads = 8
    drop_rate = 0.05

    # Training
    loss_config = {
        "name": "ce",  # ce
        "smoothing": 0.,
        "activation": "softmax",
        "aux_loss_weight": 0.,
        "activation_aux": "softmax",
    }

    data_config = {
        "batch_size": 32, #  if loss_config['name'] != "supcon" else 1024,
        "val_bs": 1024,
        "use_len_sampler": False,  # trimming is still slower, fix ?
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 1e-4,
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
    }

    epochs = 60 #if loss_config['name'] != "supcon" else 200

    use_fp16 = True

    verbose = 1
    verbose_eval = 250 #  if loss_config['name'] != "supcon" else 25

    fullfit = len(selected_folds) == 4
    n_fullfit = 1

In [ ]:
DEBUG = True
log_folder = None
run = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")
#     run = init_neptune(Config, log_folder)

df = prepare_data(DATA_PATH, Config.processed_folder)
init_distributed(Config)

preds = k_fold(Config, df, log_folder=log_folder, run=run)

Done ! 